In [67]:
import numpy as np
import os
import os.path
import pandas as pd
import cv2
import tensorflow as tf
from PIL import Image, ImageDraw

# FUNCTIONS

In [2]:
#@title Load the trained TFLite model and define some visualization functions
model_path = '/home/acarlier/code/OrnithoScope/saved_models/model_efficientDet0.tflite'

def get_classes_from_input():
    input = pd.read_csv('/home/acarlier/code/OrnithoScope/data/input.csv', names = ["split_value", "file_path", "label",
                      "x_min", "y_min", "empty_1", "empty_2",
                      "x_max", "y_max", "empty_3"])
    return input["label"].unique().tolist()

# Load the labels into a list
'''
classes = ['???'] * model.model_spec.config.num_classes
label_map = model.model_spec.config.label_map
for label_id, label_name in label_map.as_dict().items():
  classes[label_id-1] = label_name
'''
classes = get_classes_from_input()
# Define a list of colors for visualization
COLORS = np.random.randint(0, 255, size=(len(classes), 3), dtype=np.uint8)

def preprocess_image(image_path, input_size):
  """Preprocess the input image to feed to the TFLite model"""
  img = tf.io.read_file(image_path)
  img = tf.io.decode_image(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.uint8)
  original_image = img
  resized_img = tf.image.resize(img, input_size)
  resized_img = resized_img[tf.newaxis, :]
  resized_img = tf.cast(resized_img, dtype=tf.uint8)
  return resized_img, original_image


def detect_objects(interpreter, image, threshold):
  """Returns a list of detection results, each a dictionary of object info."""

  signature_fn = interpreter.get_signature_runner()

  # Feed the input image to the model
  output = signature_fn(images=image)

  # Get all outputs from the model
  count = int(np.squeeze(output['output_0']))
  scores = np.squeeze(output['output_1'])
  classes = np.squeeze(output['output_2'])
  boxes = np.squeeze(output['output_3'])

  results = []
  for i in range(count):
    if scores[i] >= threshold:
      result = {
        'bounding_box': boxes[i],
        'class_id': classes[i],
        'score': scores[i]
      }
      results.append(result)
  return results


def run_odt_and_draw_results(image_path, interpreter, threshold=0.5):
  """Run object detection on the input image and draw the detection results"""
  # Load the input shape required by the model
  _, input_height, input_width, _ = interpreter.get_input_details()[0]['shape']

  # Load the input image and preprocess it
  preprocessed_image, original_image = preprocess_image(
      image_path,
      (input_height, input_width)
    )

  # Run object detection on the input image
  results = detect_objects(interpreter, preprocessed_image, threshold=threshold)

  # Plot the detection results on the input image
  original_image_np = original_image.numpy().astype(np.uint8)
  for obj in results:
    # Convert the object bounding box from relative coordinates to absolute
    # coordinates based on the original image resolution
    ymin, xmin, ymax, xmax = obj['bounding_box']
    xmin = int(xmin * original_image_np.shape[1])
    xmax = int(xmax * original_image_np.shape[1])
    ymin = int(ymin * original_image_np.shape[0])
    ymax = int(ymax * original_image_np.shape[0])

    # Find the class index of the current object
    class_id = int(obj['class_id'])

    # Draw the bounding box and label on the image
    color = [int(c) for c in COLORS[class_id]]
    cv2.rectangle(original_image_np, (xmin, ymin), (xmax, ymax), color, 2)
    # Make adjustments to make the label visible for all objects
    y = ymin - 15 if ymin - 15 > 15 else ymin + 15
    label = "{}: {:.0f}%".format(classes[class_id], obj['score'] * 100)
    cv2.putText(original_image_np, label, (xmin, y),
        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

  # Return the final image
  original_uint8 = original_image_np.astype(np.uint8)
  return original_uint8

# Génération des prédictions

In [96]:
def get_test_images():
    input = pd.read_csv('/home/acarlier/code/OrnithoScope/data/input.csv', names = ["split_value", "file_path", "label",
                      "x_min", "y_min", "empty_1", "empty_2",
                      "x_max", "y_max", "empty_3"])
    test_dataset = input.loc[input['split_value'] == 'TEST']
    list_image_test = test_dataset["file_path"].unique().tolist() # return all unique file_path to test
    return list_image_test
loi = get_test_images()
print(loi[0])
print(len(loi))

/home/acarlier/OrnithoMate/p0133_bird_data/raw_data/task_2021-03-01_09/20210301-090002_(13.0).jpg
3516


In [108]:
def prediction_test_dataset():
    list_of_images = get_test_images()
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()
    #list_of_images = ['/home/acarlier/code/test/2021-06-11-13-44-43.jpg']
    # Load the TFLite model
    print(len(list_of_images),' to predict')
    for img in list_of_images:
        img_name = img.split('/')[-1]
        file_path = '/home/acarlier/code/output_images/output_image_efficientDet0/{}'.format(img_name)
        if os.path.exists(file_path):
            list_of_images.remove(img)
            print(len(list_of_images),' remaining')
        else:
            INPUT_IMAGE_URL = img #@param {type:"string"}
            DETECTION_THRESHOLD = 0.3 #@param {type:"number"}
            TEMP_FILE = img
            FILE_NAME = TEMP_FILE.split("/")[-1]
            #print('test1.2 : ok')
            #!wget -q -O $TEMP_FILE $INPUT_IMAGE_URL
            #!wget -q -O $INPUT_IMAGE_URL
            #print('test1.3 : ok')
            try:
                with open(TEMP_FILE) as file:
                    im = Image.open(TEMP_FILE)
                    im.thumbnail((512, 512), Image.ANTIALIAS)
                    im.save(TEMP_FILE, 'JPEG')
                    # Run inference and draw detection result on the local copy of the original file
                    detection_result_image = run_odt_and_draw_results(
                        TEMP_FILE,
                        interpreter,
                        threshold=DETECTION_THRESHOLD
                    )
                    # Show the detection result
                    #Image.fromarray(detection_result_image)
                    result = Image.fromarray(detection_result_image)
                    result.save("/home/acarlier/code/output_images/output_image_efficientDet0/{}".format(FILE_NAME), 'JPEG')
                    #result.show()
            except OSError as e:
                print(e.errno)

In [109]:
prediction_test_dataset()

3516  to predict
3515  remaining
3514  remaining
3513  remaining
3512  remaining
3511  remaining
3510  remaining
3509  remaining
3508  remaining
3507  remaining
3506  remaining
3505  remaining
3504  remaining
3503  remaining
3502  remaining
3501  remaining
3500  remaining
3499  remaining
3498  remaining
3497  remaining
3496  remaining
3495  remaining
3494  remaining
3493  remaining
3492  remaining
3491  remaining
3490  remaining
3489  remaining
3488  remaining
3487  remaining
3486  remaining
3485  remaining
3484  remaining
3483  remaining
3482  remaining
3481  remaining
3480  remaining
3479  remaining
3478  remaining
3477  remaining
3476  remaining
3475  remaining
3474  remaining
3473  remaining
3472  remaining
3471  remaining
3470  remaining
3469  remaining
3468  remaining
3467  remaining
3466  remaining
3465  remaining
3464  remaining
3463  remaining
3462  remaining
3461  remaining
3460  remaining
3459  remaining
3458  remaining
3457  remaining
3456  remaining
3455  remaining
3454  r

KeyboardInterrupt: 

# Comparaison vérité terrain

In [119]:
def comparaison_verite_terrain():
    input = pd.read_csv('/home/acarlier/code/OrnithoScope/data/input.csv', names = ["split_value", "file_path", "label",
                      "x_min", "y_min", "empty_1", "empty_2",
                      "x_max", "y_max", "empty_3"])
    input['file_path'] = input['file_path'].map(lambda x: x.split("/")[-1])
    path_output_files = '/home/acarlier/code/output_images/output_image_efficientDet0/'
    files = os.listdir(path_output_files)
    print(len(files),' to compare')
    path_comparaison_files = '/home/acarlier/code/comparaison_verite_terrain/comparaison_efficientdet0/'

    for f in files:
        path_file_compared = '{}{}'.format(path_comparaison_files,f)
        print('path to test', path_file_compared)
        if os.path.exists(path_file_compared):
            print(f, ' exists')
        else:
            print(f, ' remaining')
            input_specific_img = input[input['file_path'] == f]
            x_min_output = input_specific_img['x_min'].to_list()
            y_min_output = input_specific_img['y_min'].to_list()
            x_max_output = input_specific_img['x_max'].to_list()
            y_max_output = input_specific_img['y_max'].to_list()
            with Image.open('/home/acarlier/code/output_images/output_image_efficientDet0/{}'.format(f)) as im:
                w, h = im.size
                draw = ImageDraw.Draw(im)
                for i in range(len(x_min_output)):
                    #draw.rectangle([(x_min_output*w, y_min_output*h),(x_max_output*w, y_max_output*h)])
                    draw.rectangle([(x_min_output[i]*w,y_min_output[i]*h),(x_max_output[i]*w,y_max_output[i]*h)],outline="white")

                # write to stdout
                im.save("/home/acarlier/code/comparaison_verite_terrain/comparaison_efficientdet0/{}".format(f), "JPEG")




In [120]:
comparaison_verite_terrain()	

161  to compare
path to test /home/acarlier/code/comparaison_verite_terrain/comparaison_efficientdet0/20210301-091243_(11.0).jpg
['20210301-091243_(11.0).jpg', '20210301-090651_(6.0).jpg', '20210301-091247_(6.0).jpg', '20210301-090936_(7.0).jpg', '20210301-090925_(6.0).jpg', '20210301-090709_(8.0).jpg', '20210301-090204_(13.0).jpg', '20210301-090801_(6.0).jpg', '20210301-090209_(5.0).jpg', '20210301-091039_(6.0).jpg', '20210301-091626_(44.0).jpg', '20210301-090037_(6.0).jpg', '20210301-092456_(6.0).jpg', '20210301-092329_(25.0).jpg', '20210301-092320_(19.0).jpg', '20210301-091319_(15.0).jpg', '20210301-090227_(8.0).jpg', '20210301-090719_(7.0).jpg', '20210301-090416_(8.0).jpg', '20210301-092258_(14.0).jpg', '20210301-091639_(42.0).jpg', '20210301-091536_(21.0).jpg', '20210301-091617_(22.0).jpg', '20210301-090728_(5.0).jpg', '20210301-090618_(7.0).jpg', '20210301-090436_(17.0).jpg', '20210301-090854_(7.0).jpg', '20210301-090556_(5.0).jpg', '20210301-093515_(7.0).jpg', '20210301-091301_(